# Austin's Car Crash
#### By: Luca Comba, Hung Tran, Steven Tran

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/a/a0/Seal_of_Austin%2C_TX.svg/1024px-Seal_of_Austin%2C_TX.svg.png" width="100" height="100">

#### Table of Contents
1. [Introduction](#introduction)
2. [Data](#data)
3. [Models](#models)
4. [Conclusion](#conclusion)


# Introduction

<div id="introduction" />

The original dataset includes records of traffic accidents in Austin, Texas, from 2010 to today, with 216,088 instances and 45 features, including both numerical and categorical data. The dataset can be found at [Austin Crash Report Data](https://catalog.data.gov/dataset/vision-zero-crash-report-data).

# Data
<div id="data" />

In the following section we will set up the dataset and the features we will be using for our models.

In [1]:
# imports
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from utils.utils import (
    print_linear_regression_scores,
    backwards_elimination,
    Timer,
    Pickler
)

RANDOM_SEED=42

In [2]:
# read data
df = pd.read_csv('data/austin_car_crash_cleaned.csv')

## Data Cleaning

We went over the dataset cleansing in the file [cleaning.ipynb](./cleaning.ipynb).


## Exploratory Data Analysis

We went over the exploratory data analysis in the file [exploratory.ipynb](./exploratory.ipynb).

## Split

Before splitting the dataset into dependent and independent variables, we will drop the features that are not useful for our models.

We will be dropping the `primary_address`, `secondary_address`, `latitude`, `longitude`, and `timestamp_us_central` due to their types.

In [3]:
df = df.drop(columns=['primary_address', 'secondary_address', 'timestamp_us_central', 'latitude', 'longitude'])

In [4]:
# Define the target column
target_col = 'estimated_total_comprehensive_cost'

# Split into features (X) and target (y)
X = df.drop(columns=[target_col]).astype(int)
y = df[target_col]

In [5]:
# Split 60-20-20 train-validation-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_SEED)

## Feature Scaling

In [6]:
# Select numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

print(f"Numeric columns: {numeric_cols}")

Numeric columns: ['fatal_crash', 'speed_limit', 'construction_zone', 'sus_serious_injry_cnt', 'nonincap_injry_cnt', 'poss_injry_cnt', 'non_injry_cnt', 'unkn_injry_cnt', 'tot_injry_cnt', 'death_cnt', 'motor_vehicle_death_count', 'motor_vehicle_serious_injury_count', 'bicycle_death_count', 'bicycle_serious_injury_count', 'pedestrian_death_count', 'pedestrian_serious_injury_count', 'motorcycle_death_count', 'motorcycle_serious_injury_count', 'other_death_count', 'other_serious_injury_count', 'micromobility_serious_injury_count', 'micromobility_death_count', 'law_enforcement_fatality_count', 'severity_incapacitating_injury', 'severity_killed', 'severity_non_incapacitating_injury', 'severity_not_injured', 'severity_possible_injury', 'severity_unknown', 'unit_involved_bicycle', 'unit_involved_large_passenger_vehicle', 'unit_involved_motor_vehicle_other', 'unit_involved_motorcycle', 'unit_involved_other_unknown', 'unit_involved_passenger_car', 'unit_involved_pedestrian', 'hour', 'day_of_week'

In [7]:
cols_to_remove = [ 'unit_involved_motor_vehicle_other', 'unit_involved_motorcycle', 'unit_involved_passenger_car']

# 'unit_involved_bicycle',
# 'unit_involved_large_passenger_vehicle',
# 'unit_involved_motor_vehicle_other',
# 'unit_involved_motorcycle',
# 'unit_involved_other_unknown',
# 'unit_involved_passenger_car',
# 'unit_involved_pedestrian',


for col in cols_to_remove:
    numeric_cols.remove(col)

In [8]:
# Select categorical columns
categorical_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
print(f"Categorical columns: {categorical_cols}")

Categorical columns: []


In [9]:
print(f'Train after scaler. mean: {np.mean(X_train[numeric_cols], axis=0)} std: {np.std(X_train[numeric_cols], axis=0)}')
print(f'Test after scaler. mean: {np.mean(X_test[numeric_cols], axis=0)} std: {np.std(X_test[numeric_cols], axis=0)}')

Train after scaler. mean: fatal_crash                                 0.005610
speed_limit                                45.911993
construction_zone                           0.051477
sus_serious_injry_cnt                       0.034929
nonincap_injry_cnt                          0.284557
poss_injry_cnt                              0.358758
non_injry_cnt                               1.848982
unkn_injry_cnt                              0.109255
tot_injry_cnt                               0.678243
death_cnt                                   0.005788
motor_vehicle_death_count                   0.002899
motor_vehicle_serious_injury_count          0.024441
bicycle_death_count                         0.000136
bicycle_serious_injury_count                0.001790
pedestrian_death_count                      0.001874
pedestrian_serious_injury_count             0.003517
motorcycle_death_count                      0.000879
motorcycle_serious_injury_count             0.005171
other_death_count   

In [10]:
# Scale numeric features
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_val[numeric_cols] = scaler.transform(X_val[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [11]:
print(f'Train after scaler. mean: {np.mean(X_train[numeric_cols], axis=0)} std: {np.std(X_train[numeric_cols], axis=0)}')
print(f'Test after scaler. mean: {np.mean(X_test[numeric_cols], axis=0)} std: {np.std(X_test[numeric_cols], axis=0)}')

Train after scaler. mean: fatal_crash                             -1.175090e-17
speed_limit                             -1.463656e-16
construction_zone                       -6.202690e-17
sus_serious_injry_cnt                   -1.517205e-17
nonincap_injry_cnt                      -7.467028e-17
poss_injry_cnt                           6.098569e-17
non_injry_cnt                           -5.079661e-17
unkn_injry_cnt                          -1.204839e-17
tot_injry_cnt                           -6.277063e-17
death_cnt                               -2.000628e-17
motor_vehicle_death_count                3.837636e-17
motor_vehicle_serious_injury_count      -4.209500e-17
bicycle_death_count                      4.239249e-18
bicycle_serious_injury_count             1.896506e-18
pedestrian_death_count                   5.726705e-18
pedestrian_serious_injury_count          1.502330e-17
motorcycle_death_count                  -2.766668e-17
motorcycle_serious_injury_count          4.060754e-17
ot

## Feature Selection
<div id="feature-selection" />

We ran the [feature_selection.ipynb](./feature_selection.ipynb) notebook for a preview, and now we will need to drop some features from the dataset.

After the data split we will be using the **Backward Elimination** method for dropping unsatisfactory features from the dataset.

In [12]:
backwards_eliminated_features = backwards_elimination(X_train, y_train)
print(f"Removed features: {backwards_eliminated_features}")

Removed features: ['month_sin', 'construction_zone', 'motor_vehicle_death_count', 'severity_killed', 'pedestrian_serious_injury_count', 'other_serious_injury_count', 'micromobility_serious_injury_count', 'month', 'bicycle_serious_injury_count', 'severity_not_injured', 'motorcycle_serious_injury_count', 'motor_vehicle_serious_injury_count', 'unit_involved_large_passenger_vehicle', 'severity_incapacitating_injury', 'sus_serious_injry_cnt', 'day_of_month', 'unit_involved_bicycle', 'hour_sin', 'law_enforcement_fatality_count', 'day_of_week', 'unit_involved_other_unknown', 'year', 'unit_involved_passenger_car', 'month_cos', 'bicycle_death_count', 'unit_involved_motorcycle']


In [13]:
print(f"Features before backwards elimination: {len(df.columns)}")

cols = df.columns.tolist()
df = df.drop(columns=backwards_eliminated_features)

print(f"Features after backwards elimination: {len(df.columns)}")

Features before backwards elimination: 47
Features after backwards elimination: 21


In [14]:
display(df.columns)

Index(['fatal_crash', 'speed_limit', 'nonincap_injry_cnt', 'poss_injry_cnt',
       'non_injry_cnt', 'unkn_injry_cnt', 'tot_injry_cnt', 'death_cnt',
       'pedestrian_death_count', 'motorcycle_death_count', 'other_death_count',
       'micromobility_death_count', 'estimated_total_comprehensive_cost',
       'severity_non_incapacitating_injury', 'severity_possible_injury',
       'severity_unknown', 'unit_involved_motor_vehicle_other',
       'unit_involved_pedestrian', 'hour', 'weekend', 'hour_cos'],
      dtype='object')

# Models
<div id="models" />
We will be using the following models:

To predict the `estimated_total_comprehensive_cost` we can use:
1. Linear Regression
2. Ridge Regression
3. Lasso Regression
4. Decision Tree Regression
5. Random Forest Regression
6. Support Vector Regression (SVR)
7. K-Nearest Neighbors (KNN) Regression

## Pipeline

The sklearn pipeline will help us creating models.

In [15]:
preprocessor = None # No preprocessing needed yet, usually we can run feature scaling and feature selection here

In [16]:
models = {
    'linear_regression': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ]),
    "ridge_regression": Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Ridge(alpha=1.0))
    ]),
    "lasso_regression": Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Lasso(alpha=0.1))
    ]),
    "decision_tree_regressor": Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', DecisionTreeRegressor(random_state=RANDOM_SEED))
    ]),
    "random_forest_regressor": Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ]),
    # SVR needs a better hardware
    "svr": Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', SVR(kernel='rbf', C=1.0, epsilon=0.1))
    ]),
    "k_neighbors_regressor": Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', KNeighborsRegressor(n_neighbors=90, n_jobs=-1))
    ])
}

### Validation Set

On the validation set we will be training the models and then we will be using the test set to evaluate the models.

In [17]:

validation_results = {}
for model_name, model_pipeline in models.items():
    print(f"Training {model_name}")
    with Timer():
        model_pipeline.fit(X_train, y_train)

    y_hat = model_pipeline.predict(X_val)

    validation_results[model_name] = y_hat

    metrics_dict = print_linear_regression_scores(model_name, y_val, y_hat)
    print()
    validation_results[model_name] = {"y_hat": y_hat, **metrics_dict}

Training linear_regression
Executed in 0.1418 seconds
linear_regression Scores:
MAE: 596.9370 MSE: 6163872.5262 RMSE: 2482.7147 RMSE%: 0.81% R^2: 1.0000

Training ridge_regression
Executed in 0.0424 seconds
ridge_regression Scores:
MAE: 597.4020 MSE: 6163891.9397 RMSE: 2482.7187 RMSE%: 0.81% R^2: 1.0000

Training lasso_regression
Executed in 0.3491 seconds
lasso_regression Scores:
MAE: 600.0804 MSE: 6233238.1675 RMSE: 2496.6454 RMSE%: 0.82% R^2: 1.0000

Training decision_tree_regressor
Executed in 0.3333 seconds
decision_tree_regressor Scores:
MAE: 1041.5751 MSE: 2378662940.4007 RMSE: 48771.5382 RMSE%: 15.98% R^2: 0.9953

Training random_forest_regressor
Executed in 15.9452 seconds
random_forest_regressor Scores:
MAE: 903.7116 MSE: 1007878969.4153 RMSE: 31747.1096 RMSE%: 10.40% R^2: 0.9980

Training svr
Executed in 503.1431 seconds
svr Scores:
MAE: 258236.7713 MSE: 556524423556.0162 RMSE: 746005.6458 RMSE%: 244.50% R^2: -0.0886

Training k_neighbors_regressor
Executed in 0.0360 seconds

### Grid Search for hyperparameters tuning

We will be using the `GridSearchCV` method to find the best hyperparameters for our models.

In [18]:
param_grids = {
    'linear_regression': {},
    'ridge_regression': {
        'regressor__alpha': [0.1, 1.0, 10.0]
    },
    'lasso_regression': {
        'regressor__alpha': [0.1, 1.0, 10.0]
    }, 
    'decision_tree_regressor': {
        'regressor__max_depth': [3,5,10],                         
        'regressor__min_samples_split':[2,5,10]
    },
    'random_forest_regressor': {
        'regressor__n_estimators':[50, 100, 200],
        'regressor__max_depth':[3,5,10]
    }, 
    'srv': {
        'regressor__C': [0.1, 1.0, 10.0], # penalty parameter
        'regressor__epsilon': [0.1, 0.2, 0.5], # no pentaly if error is within epsilon
        'regressor__kernel': ['linear', 'rbf']
    }, 
    'k_neighbors_regressor': {
        'regressor__n_neighbors': [3,5,10], # how many neighbors to look at 
        'regressor__weights': ['uniform', 'distance'] # how to weight neighbors
    }
}

In [19]:
models_with_gridsearch = {}

for model_name, model_pipeline in models.items(): 
    param_grid = param_grids.get(model_name, {})
    
    grid_search = GridSearchCV(
        estimator = model_pipeline, 
        param_grid = param_grid, 
        cv = 5, 
        n_jobs = -1, 
        scoring = 'neg_mean_squared_error'
    )
    
    models_with_gridsearch[model_name] = grid_search

print(f"Models with grid search:\n {models_with_gridsearch}")

Models with grid search:
 {'linear_regression': GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor', None),
                                       ('regressor', LinearRegression())]),
             n_jobs=-1, param_grid={}, scoring='neg_mean_squared_error'), 'ridge_regression': GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor', None),
                                       ('regressor', Ridge())]),
             n_jobs=-1, param_grid={'regressor__alpha': [0.1, 1.0, 10.0]},
             scoring='neg_mean_squared_error'), 'lasso_regression': GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor', None),
                                       ('regressor', Lasso(alpha=0.1))]),
             n_jobs=-1, param_grid={'regressor__alpha': [0.1, 1.0, 10.0]},
             scoring='neg_mean_squared_error'), 'decision_tree_regressor': GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor', None),
                      

In [20]:
hyperparameter_results = {}
for model_name, grid_search in models_with_gridsearch.items(): 
    print(f"Fitting {model_name} with GridSearchCV")
    with Timer():
        grid_search.fit(X_train, y_train)

    y_hat = grid_search.predict(X_val)

    metrics_dict = print_linear_regression_scores(model_name, y_val, y_hat)
    print(f'Best params for {model_name}: {grid_search.best_params_}')
    print(f'Best score for {model_name}: {grid_search.best_score_}')
    print()
    hyperparameter_results[model_name] = {
        "y_hat": y_hat, 
        "params": grid_search.best_params_, 
        "scores": grid_search.best_score_, 
        **metrics_dict
    }


Fitting linear_regression with GridSearchCV
Executed in 3.5117 seconds
linear_regression Scores:
MAE: 596.9370 MSE: 6163872.5262 RMSE: 2482.7147 RMSE%: 0.81% R^2: 1.0000
Best params for linear_regression: {}
Best score for linear_regression: -26389149.68813243

Fitting ridge_regression with GridSearchCV
Executed in 1.8320 seconds
ridge_regression Scores:
MAE: 597.4020 MSE: 6163891.9397 RMSE: 2482.7187 RMSE%: 0.81% R^2: 1.0000
Best params for ridge_regression: {'regressor__alpha': 1.0}
Best score for ridge_regression: -26387326.494386908

Fitting lasso_regression with GridSearchCV
Executed in 2.3009 seconds
lasso_regression Scores:
MAE: 603.6336 MSE: 6237030.9737 RMSE: 2497.4048 RMSE%: 0.82% R^2: 1.0000
Best params for lasso_regression: {'regressor__alpha': 10.0}
Best score for lasso_regression: -5402169.726188124

Fitting decision_tree_regressor with GridSearchCV
Executed in 3.9369 seconds
decision_tree_regressor Scores:
MAE: 1323.0941 MSE: 584700146.6969 RMSE: 24180.5737 RMSE%: 7.92% 

### Hyperparameter tuning

We will be using the `GridSearchCV` method to find the best hyperparameters for our models.

In [21]:
# Example of setting a hyperparameter for a model
# models['k_neighbors_regressor'].set_params(regressor__n_neighbors=200)

for model_name in hyperparameter_results:
    for param_name, param_value in hyperparameter_results[model_name]['params'].items():
        print(f"Setting {model_name} {param_name} to {param_value}")
        models[model_name].set_params(**{param_name: param_value})

Setting ridge_regression regressor__alpha to 1.0
Setting lasso_regression regressor__alpha to 10.0
Setting decision_tree_regressor regressor__max_depth to 10
Setting decision_tree_regressor regressor__min_samples_split to 5
Setting random_forest_regressor regressor__max_depth to 10
Setting random_forest_regressor regressor__n_estimators to 200
Setting k_neighbors_regressor regressor__n_neighbors to 3
Setting k_neighbors_regressor regressor__weights to distance


### Testing Set

Now we can adjust the models to the test set and evaluate them.

In [22]:
testing_results = {}

for model_name, model_pipeline in models.items():
    print(f"Training {model_name}")

    with Timer():
        model_pipeline.fit(X_train, y_train)

    y_hat = model_pipeline.predict(X_test)

    testing_results[model_name] = y_hat

    metrics_dict = print_linear_regression_scores(model_name, y_test, y_hat)
    print()
    testing_results[model_name] = {"y_hat": y_hat, **metrics_dict}

Training linear_regression
Executed in 0.1368 seconds
linear_regression Scores:
MAE: 567.2117 MSE: 4126473.4143 RMSE: 2031.3723 RMSE%: 0.65% R^2: 1.0000

Training ridge_regression
Executed in 0.1048 seconds
ridge_regression Scores:
MAE: 567.7651 MSE: 4127465.3063 RMSE: 2031.6164 RMSE%: 0.65% R^2: 1.0000

Training lasso_regression
Executed in 0.3092 seconds
lasso_regression Scores:
MAE: 577.0529 MSE: 4196869.4865 RMSE: 2048.6262 RMSE%: 0.66% R^2: 1.0000

Training decision_tree_regressor
Executed in 0.3100 seconds
decision_tree_regressor Scores:
MAE: 2319.3195 MSE: 7758868954.3191 RMSE: 88084.4422 RMSE%: 28.16% R^2: 0.9871

Training random_forest_regressor
Executed in 26.0065 seconds
random_forest_regressor Scores:
MAE: 2188.0785 MSE: 8259133836.7700 RMSE: 90879.7768 RMSE%: 29.06% R^2: 0.9862

Training svr
Executed in 491.3659 seconds
svr Scores:
MAE: 266270.5423 MSE: 648597134004.6821 RMSE: 805355.2843 RMSE%: 257.50% R^2: -0.0811

Training k_neighbors_regressor
Executed in 0.0177 second

# Conclusion
<div id="conclusion" />

In this project we have gone over the dataset and created a model to predict the `estimated_total_comprehensive_cost` of a car crash in Austin, Texas. We have used different models and evaluated them using the test set. The best model was the Random Forest Regression with an R2 score of 0.85.

### Saving results

In [23]:
data = (validation_results, hyperparameter_results, testing_results)

filename = './data/notebook_data.pkl'
Pickler(filename).save(data)